In [ ]:
from typing import List, Optional

import fire

from llama import Dialog, Llama
import torch.distributed as dist
import torch
import os
from data_casual import output_list_test, input_list_test
from llama.model import ModelArgs, Transformer
from llama.tokenizer import ChatFormat, Dialog, Message, Tokenizer
import json
import os
import sys
import time
from pathlib import Path
from typing import List, Optional, Tuple, TypedDict

import torch
import torch.nn.functional as F
from fairscale.nn.model_parallel.initialize import (
    get_model_parallel_rank,
    initialize_model_parallel,
    model_parallel_is_initialized,
)

In [ ]:
steering_vec = torch.load("./steering_vector.pt")
steering_vec_2 = torch.load("./steering_vector_1.pt")
steering_vec_3 = torch.load("./steering_vector_2.pt")
steering_vec_layer_28 = torch.load("./steering_vector_layer28.pt")


In [ ]:
steering_vec_layer_28

In [ ]:
generator = Llama.build("./", "./tokenizer.model",max_seq_len= 1024,max_batch_size= 4, activation=True, activation_layer=28,steering_vector=steering_vec_layer_28)

In [ ]:
ckpt_dir = "./"
tokenizer_path = "./tokenizer.model"
temperature = 0.6
top_p = 0.9
max_seq_len = 1024
max_batch_size = 4
max_gen_len= generator.model.params.max_seq_len - 1

In [ ]:
generator.model.adding_activation_vector = 1.2*steering_vec_layer_28

In [ ]:
generator.model.adding_activation_vector

In [ ]:
value, activations = generator.chat_completion([input_list_test[1142]],
                                   max_gen_len=max_gen_len,
                                   top_p=top_p,
                                   temperature=temperature)
print(value, activations)

n = len(input_list_test)
loss = 0
for i in range(n):
    value, activations = generator.chat_completion([input_list_test[i]],
                                      max_gen_len=max_gen_len,
                                       top_p=top_p,
                                       temperature=temperature)
    value2 = 1 if value[0]["generation"]["content"]=="T" else 0
    loss += 1/n*abs(int(output_list_test[i])-int(value2))

print(loss)